In [1]:
import sympy as sym
import math as m
import numpy as np
import pandas as pd


import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize


import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import SIC_POVM_functions as sic

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


[(6+0j), (6+0j), (2.4+0j), (2.5999999999999996+1.637849848627056e-17j), (2.287500258169+0j), (2.4+2.2311252215006176e-17j), (2.4-2.576281494458285e-17j), (2.3000000000000003-4.125283626344832e-18j), (2.587500258169+0j)]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:239: ComplexWarning: Casting complex values to real discards the imaginary part
  retval = _minpack._hybrd(func, x0, args, 1, xtol, maxfev,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [2]:
#Class definitions
class seed_sol:
    def __init__(self, seed):
        self.seed = seed
    @property
    def initial_guess(self):
        np.random.seed(self.seed)
        initial_guess = np.random.rand(35)  # random initial guess
        return initial_guess
    @property
    def solution(self):
        solution = fsolve(Lagrange_eqs, self.initial_guess)
        return solution
    @property
    def res_list(self):
        res_list = Lagrange_eqs(self.solution)
        return res_list
    @property
    def res(self):
        res = np.sum(np.abs(self.res_list))
        return res


class POVM_relations:
    def __init__(self, POVM_vec_list):
        self.POVM_vec_list = POVM_vec_list
        self.inner_products = {}
    
    def dot_product(self, i, j):
        return np.vdot(self.POVM_vec_list[i], self.POVM_vec_list[j])

    @property
    def all(self):
        if len(self.POVM_vec_list) != 9:
            raise ValueError('The POVM list must have 9 elements')
        for i in range(9):
            for j in range(i, 9):
                key = f'{i+1}{j+1}'
                value = self.dot_product(i, j)
                self.inner_products[key] = value
                # if abs(value) < 1e-18:
                    # self.inner_products[key] = 0
        return self.inner_products

In [7]:
# creating five lists POVMs
w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_unnormalized = [[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]             # unnormalized POVM direction vectors
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors

c4j_list = [2,.5,.5,.5, (-.25-.433013j), -.25, .5, -.25, (-.25-.433013j)]           # fourth elements, not normalized yet. See sic.py for calcs
c5j_list = [0, np.sqrt(15)/2 , 0.38729833462074165, (-0.38729833462074176-0.44721359549995776j), -0.19364916731037082, 0.5809475019311124, -0.38729833462074165, (-0.19364916731037093-0.4472135954999579j), 0.5809475019311124]

POVM_unnormalized_5d = []
for i in range(len(POVM_unnormalized)):
	# Directly append c4j_list[i] and c5j_list[i] to the copies of lists in POVM_unnormalized
	vec_5d_i = POVM_unnormalized[i] + [c4j_list[i], c5j_list[i]]
	POVM_unnormalized_5d.append(vec_5d_i)

POVM_unnormalized_5d[0]    ## unnormalized 5-lists POVM vectors

[0, 1, -1, 2, 0]

In [10]:
# delta_five= [0.400000000000000, 0.433333333333333, 0.381250043028167, 0.400000000000000, 0.400000000000000, 0.383333333333333, 0.431250043028167]
delta_five  = [0.400000000000000, 0.433333333333333, 0.381250043028167, 0.400000000000000, 0.400000000000000, 0.383333333333333, 0.431250043028167]

In [11]:
def Lagrange_eqs(vars):
    # Unpacking variables, C63 to C99 and y3 to y9
    C63, C64, C65, C66, C67, C68, C69,  C73, C74, C75, C76, C77, C78, C79,  C83, C84, C85, C86, C87, C88, C89,  C93, C94, C95, C96, C97, C98, C99,  y3, y4, y5, y6, y7, y8, y9 = vars
    global delta_five
    delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five

    # Full set of 28 equations for Cij terms, based on previous interactions
    equations = [
        # Derivatives with respect to C63 to C69
        np.conj(C63)*y3, np.conj(C63) + np.conj(C64)*y4, np.conj(C63) + np.conj(C64) + np.conj(C65)*y5, np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66)*y6,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67)*y7,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68)*y8,
        np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68) + np.conj(C69)*y9,
        # Continue with C73 to C79
        
        # Derivatives with respect to C73 to C79
        np.conj(C73)*y3, np.conj(C73) + np.conj(C74)*y4, np.conj(C73) + np.conj(C74) + np.conj(C75)*y5, np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76)*y6,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77)*y7,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78)*y8,
        np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78) + np.conj(C79)*y9,

        # Derivatives with respect to C83 to C89
        np.conj(C83)*y3, np.conj(C83) + np.conj(C84)*y4, np.conj(C83) + np.conj(C84) + np.conj(C85)*y5, np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86)*y6,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87)*y7,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88)*y8,
        np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88) + np.conj(C89)*y9,
        # Derivatives with respect to C93 to C99
        np.conj(C93)*y3, np.conj(C93) + np.conj(C94)*y4, np.conj(C93) + np.conj(C94) + np.conj(C95)*y5, np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96)*y6,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97)*y7,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98)*y8,
        np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98) + np.conj(C99)*y9,
        #Now the normalization equations
        abs(C63)**2 + abs(C73)**2 + abs(C83)**2 + abs(C93)**2 + delta_33 - 1,abs(C64)**2 + abs(C74)**2 + abs(C84)**2 + abs(C94)**2 + delta_44 - 1,
        abs(C65)**2 + abs(C75)**2 + abs(C85)**2 + abs(C95)**2 + delta_55 - 1,abs(C66)**2 + abs(C76)**2 + abs(C86)**2 + abs(C96)**2 + delta_66 - 1,
        abs(C67)**2 + abs(C77)**2 + abs(C87)**2 + abs(C97)**2 + delta_77 - 1,abs(C68)**2 + abs(C78)**2 + abs(C88)**2 + abs(C98)**2 + delta_88 - 1,
        abs(C69)**2 + abs(C79)**2 + abs(C89)**2 + abs(C99)**2 + delta_99 - 1
    ]

    assert len(equations) == 35
    return equations

# Example Usage:
initial_guess = [0.2] * 15 + [-.2]*15 + [0.1] * 5   # Initial guess for the variables (Cij real and imaginary parts, y)
solution = fsolve(Lagrange_eqs, initial_guess)
residuals = Lagrange_eqs(solution)                   # residuals and residues sum
residuals_sum = np.sum(np.abs(residuals))

print("Solution to the system:", solution)
print("Residuals:", residuals)
print("Residuals sum:", residuals_sum)

Solution to the system: [ 1.67970220e-02 -1.63237712e-02  1.70574559e-02 -1.67970220e-02
 -1.67970220e-02  1.70287160e-02  1.63537502e-02  1.68243798e-02
 -1.63503582e-02  1.70852378e-02 -1.68243798e-02 -1.68243798e-02
  1.70564512e-02  1.63803860e-02  7.74044343e-01 -7.52235888e-01
  7.86045719e-01 -7.74044343e-01 -7.74044343e-01  7.84721321e-01
  7.53617378e-01  1.70339505e-02 -1.65540244e-02  1.72980579e-02
 -1.70339505e-02 -1.70339504e-02  1.72689125e-02  1.65844257e-02
 -6.12712937e-25  1.02899151e+00 -2.77445120e-02  1.04367945e+00
  4.36794495e-02  9.43308780e-01 -5.90310284e-02]
Residuals: [-1.0291752664914375e-26, 3.642919299551295e-16, -7.043804016622357e-13, -2.1128713362239537e-11, -2.0709264056222032e-11, -2.4828090972039973e-12, -1.1790536754395509e-11, -1.0308515143749554e-26, 3.5388358909926865e-16, -7.004946210760477e-13, -2.1012247497509406e-11, -2.0588722783945745e-11, -2.467883536416693e-12, -1.1720086056168921e-11, -4.742669829629579e-25, 4.440892098500626e-16, -2.

In [7]:
# Example Usage:
for i in range(10):
    np.random.seed(i)
    initial_guess = np.concatenate(([0], np.random.rand(6), [0], np.random.rand(6), [0], np.random.rand(6), [0], np.random.rand(6), np.random.rand(7)))

    solution = fsolve(Lagrange_eqs, initial_guess)
    residuals = Lagrange_eqs(solution)                   # residuals and residues sum
    residuals_sum = np.sum(np.abs(residuals))

    if residuals_sum < 1e-13:
        print("Seed:", i)
        print("Residuals sum:", residuals_sum)
        print("Solution to the system:", solution)

# print("Solution to the system:", solution)
# print("Residuals:", residuals)
# print("Residuals sum:", residuals_sum)

Working. Now, conducting numerous trials using random initial points and filtering out the best outcomes. Then, converting those results into class objects for analysis.

In [ ]:
seed_res_list = []
for i in range(1500):
    np.random.seed(i)
    initial_guess = np.random.rand(35)  # random initial guess
    solution = fsolve(Lagrange_eqs, initial_guess)
    residuals = Lagrange_eqs(solution)
    residuals_sum = np.sum(np.abs(residuals))
    if residuals_sum < 1e-11:
        seed_res_list.append((residuals_sum, i))
    seed_res_list.sort()            # sorted in order of increasing residue sum

seed_list = [i[1] for i in seed_res_list]   # create seed list for later use in class objects
sol_list = [seed_sol(i) for i in seed_list]  # storing class objects, list containing all good solutions as class instances

In [ ]:
# doing some clustering to know if the points are close by or far especially the lambda points


points = []
for i in range(len(sol_list)):
    lambdas_point = sorted(abs(sol_list[i].solution()[-7:]))
    points.append(lambdas_point)

data = points  
# Using the Elbow Method to find the optimal number of clusters
wcss = []  # Within-cluster sum of squares
for i in range(1, 11):
	kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
	kmeans.fit(data)
	wcss.append(kmeans.inertia_)

# Plotting the results of the Elbow Method
plt.figure(figsize=(10, 7))
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')  # Within cluster sum of squares
plt.show()

# Choose the number of clusters based on the elbow point from the plot
# For this example, let's assume the optimal number is 3 (this should be chosen based on the plot)
optimal_clusters = 5
kmeans_optimal = KMeans(n_clusters=optimal_clusters, init='k-means++', max_iter=300, n_init=10, random_state=42)
kmeans_optimal.fit(data)

# Reducing dimensions for visualization using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(data)

# Plotting the results
plt.figure(figsize=(10, 7))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans_optimal.labels_, cmap='viridis', edgecolor='k', s=100)
plt.title("Visualization of 9D data in 2D using PCA")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.colorbar(label='Cluster Label')
plt.show()

# Printing the number of centers
print(f"Number of centers (clusters) found: {optimal_clusters}")

# After fitting the KMeans model to the data
labels = kmeans_optimal.labels_

# Count the number of points in each cluster
cluster_counts = np.bincount(labels)

# Print the counts
for i, count in enumerate(cluster_counts):
	print(f"Cluster {i}: {count} points")



In [22]:
#verifying a solution by checking dot product relations, say for seed = 1254
sol_1254 = seed_sol(1254).solution
solution = sol_1254
# adding to the 5-lists POVM vectors, the first 28 elements of the solution, position wise

POVM_unnormalized_9d = []
POVM_unnormalized_9d = [i for i in POVM_unnormalized_5d]
POVM_unnormalized_9d[0] = POVM_unnormalized_5d[0]+ [0,0,0,0]
POVM_unnormalized_9d[1] = POVM_unnormalized_5d[1]+ [0,0,0,0]


for j in range(4):                  # kinda segementing the 28 elements into 4 groups of 7 elements each
    for i in range(7):
        POVM_unnormalized_9d[i+2] = POVM_unnormalized_5d[i+2] + [solution[7*j + i]]      # iteratively adding the 7 solutions to the 5-lists POVM vectors over four iterations

print(POVM_unnormalized_9d[0]    )
print(POVM_unnormalized_9d[1]    )
print(POVM_unnormalized_9d[2]    )
print(POVM_unnormalized_9d[3]    )


print([len(i) for i in POVM_unnormalized_9d])



# POVM_9D = (1/np.sqrt(6))*np.array(POVM_unnormalized_9d)   # normalizing the 9D POVM vectors by dividing by sqrt(6), see notes
# POVM_9D[0]


[0, 1, -1, 2, 0, 0, 0, 0, 0]
[-1, 0, 1, 0.5, 1.9364916731037085, 0, 0, 0, 0]
[1, -1, 0, 0.5, 0.38729833462074165, 0.41811259608007384]
[0, (-0.49999999999999983+0.8660254037844387j), (0.5000000000000002+0.8660254037844384j), 0.5, (-0.38729833462074176-0.44721359549995776j), -0.40633240572863255]
[9, 9, 6, 6, 6, 6, 6, 6, 6]


In [25]:
sol_1254 = seed_sol(1254).solution
solution = sol_1254
#break solution into 4 parts, remove the last 7 elements
solution_add = [solution[:7], solution[7:14], solution[14:21], solution[21:28]]     # 4 parts of the solution

# Extended list creation
POVM_unnormalized_9d = [list(i) for i in POVM_unnormalized_5d]         # deep copy 

# Fill the first two vectors with zeros to make them 9D
for i in range(2):
    POVM_unnormalized_9d[i].extend([0] *4)

for part in solution_add:
    for i in range(7):
        POVM_unnormalized_9d[i+2].append(part[i])



# print(POVM_unnormalized_5d[0])
# print(POVM_unnormalized_5d[1])
# print(POVM_unnormalized_5d[2])

# print(POVM_unnormalized_9d[0])
# print(POVM_unnormalized_9d[1])
print(POVM_unnormalized_9d[2])
print(POVM_unnormalized_9d[3])


[1, -1, 0, 0.5, 0.38729833462074165, 0.2999465851496958, 0.32615612749817663, 0.4783681464642169, 0.41811259608007384]
[0, (-0.49999999999999983+0.8660254037844387j), (0.5000000000000002+0.8660254037844384j), 0.5, (-0.38729833462074176-0.44721359549995776j), -0.2914956848384994, -0.3169667816084832, -0.4648902750098601, -0.40633240572863255]


In [24]:
solution[:28]

array([ 0.29994659, -0.29149568, -0.30459719,  0.29994659,  0.29994659,
       -0.30408397, -0.29203102,  0.32615613, -0.31696678, -0.3312131 ,
        0.32615613,  0.32615613, -0.33065505, -0.31754889,  0.47836815,
       -0.46489028, -0.48578513,  0.47836815,  0.47836815, -0.48496664,
       -0.46574405,  0.4181126 , -0.40633241, -0.42459533,  0.4181126 ,
        0.4181126 , -0.42387994, -0.40707864])